# Universidade de Fortaleza
## Mestrado em Informática Aplicada
### Ciência de Dados aplicada à Ciência da Cidade

# Análise de Homicídios vs População por Município (População ≥ 10.000)

Este notebook analisa a relação entre homicídios e população dos municípios brasileiros para os anos: 1980, 1991, 2000, 2010 e 2022.

**FILTRO**: Apenas municípios com população igual ou superior a 10.000 habitantes são considerados nesta análise.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## 1. Carregamento dos Dados

In [ ]:
# Carregar dados de homicídios (descartando a primeira linha)
homicides_df = pd.read_csv('homicidios_por_municipios_1980_2024.csv', 
                           sep=';', 
                           skiprows=1,
                           encoding='utf-8-sig')

# Carregar dados de população por município (descartando a primeira linha)
population_df = pd.read_csv('populacao_por_municipio_1980_2022.csv', 
                           sep=';', 
                           skiprows=1,
                           encoding='utf-8-sig')

print("Dimensões dos dados de homicídios:", homicides_df.shape)
print("Dimensões dos dados de população:", population_df.shape)
print("\nPrimeiras linhas - Homicídios:")
display(homicides_df.head())
print("\nPrimeiras linhas - População:")
display(population_df.head())

## 2. Processamento dos Dados para Anos Específicos

In [ ]:
# Anos de interesse
years = [1980, 1991, 2000, 2010, 2022]

# Converter colunas de anos para string (como estão nos CSVs)
year_cols = [str(year) for year in years]

# Converter para numérico
for year in year_cols:
    if year in homicides_df.columns:
        homicides_df[year] = pd.to_numeric(homicides_df[year], errors='coerce')
    if year in population_df.columns:
        population_df[year] = pd.to_numeric(population_df[year], errors='coerce')

In [ ]:
# Mesclar dados de homicídios e população por código de município
merged_df = homicides_df[['Código', 'Município', 'Sigla'] + year_cols].merge(
    population_df[['Código'] + year_cols],
    on='Código',
    suffixes=('_homicides', '_population'),
    how='inner'
)

print("Dimensões dos dados mesclados:", merged_df.shape)
display(merged_df.head())

## 3. Análise de Homicídios vs População por Ano

In [ ]:
# Criar figura com subplots para cada ano
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# FILTRO: População mínima
MIN_POPULATION = 10000

# Loop através dos anos
for idx, year in enumerate(years):
    year_str = str(year)
    
    # Colunas correspondentes
    homicides_col = f"{year_str}_homicides"
    population_col = f"{year_str}_population"
    
    # Filtrar dados válidos (sem NaN, população >= 10000, homicídios > 0)
    mask = (merged_df[homicides_col].notna() & 
            merged_df[population_col].notna() & 
            (merged_df[population_col] >= MIN_POPULATION) &
            (merged_df[homicides_col] > 0))
    
    data_year = merged_df[mask].copy()
    
    # Scatter plot
    axes[idx].scatter(data_year[population_col], 
                     data_year[homicides_col],
                     alpha=0.5, s=20)
    
    axes[idx].set_xlabel('População', fontsize=10)
    axes[idx].set_ylabel('Homicídios', fontsize=10)
    axes[idx].set_title(f'Ano {year}\n(n={len(data_year)} municípios, pop ≥ {MIN_POPULATION:,})', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    
    print(f"\nAno {year}: {len(data_year)} municípios com dados válidos (pop ≥ {MIN_POPULATION:,})")

# Remover subplot extra
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

## 4. Análise em Escala Log-Log

In [ ]:
# Criar figura com subplots em escala log-log
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# FILTRO: População mínima
MIN_POPULATION = 10000

# Dicionário para armazenar dados processados por ano
data_by_year = {}

# Loop através dos anos
for idx, year in enumerate(years):
    year_str = str(year)
    
    # Colunas correspondentes
    homicides_col = f"{year_str}_homicides"
    population_col = f"{year_str}_population"
    
    # Filtrar dados válidos (sem NaN, população >= 10000, homicídios > 0)
    mask = (merged_df[homicides_col].notna() & 
            merged_df[population_col].notna() & 
            (merged_df[population_col] >= MIN_POPULATION) &
            (merged_df[homicides_col] > 0))
    
    data_year = merged_df[mask].copy()
    
    # Armazenar dados
    data_by_year[year] = {
        'population': data_year[population_col].values,
        'homicides': data_year[homicides_col].values,
        'municipalities': data_year['Município'].values,
        'states': data_year['Sigla'].values
    }
    
    # Scatter plot em escala log-log
    axes[idx].loglog(data_year[population_col], 
                     data_year[homicides_col],
                     'o', alpha=0.5, markersize=4)
    
    axes[idx].set_xlabel('População (log)', fontsize=10)
    axes[idx].set_ylabel('Homicídios (log)', fontsize=10)
    axes[idx].set_title(f'Ano {year} - Escala Log-Log\n(n={len(data_year)} municípios, pop ≥ {MIN_POPULATION:,})', 
                       fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, which='both')

# Remover subplot extra
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

print(f"\nDados armazenados no dicionário 'data_by_year' (pop ≥ {MIN_POPULATION:,}).")

## 5. Preparação para Interpolação Linear em Log-Log

Os dados estão preparados para interpolação linear em escala log-log (filtrados para população ≥ 10.000).

In [ ]:
# Visualizar estrutura dos dados preparados
for year in years:
    print(f"\nAno {year}:")
    print(f"  - Número de municípios: {len(data_by_year[year]['population'])}")
    print(f"  - População: min={data_by_year[year]['population'].min():.0f}, max={data_by_year[year]['population'].max():.0f}")
    print(f"  - Homicídios: min={data_by_year[year]['homicides'].min():.0f}, max={data_by_year[year]['homicides'].max():.0f}")

## 6. Interpolação Linear em Log-Log

Interpolação linear em escala log-log para municípios com população ≥ 10.000 habitantes.

In [ ]:
# Interpolação linear em escala log-log para cada ano
from scipy import stats

# Armazenar resultados da regressão
regression_results = {}

# Criar figura com subplots para visualizar as regressões
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, year in enumerate(years):
    # Obter dados do ano
    pop = data_by_year[year]['population']
    hom = data_by_year[year]['homicides']
    
    # Transformar para escala logarítmica
    log_pop = np.log10(pop)
    log_hom = np.log10(hom)
    
    # Regressão linear: log(homicides) = alpha + beta * log(population)
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_pop, log_hom)
    
    # Armazenar resultados
    regression_results[year] = {
        'beta': slope,
        'alpha': intercept,
        'r_squared': r_value**2,
        'p_value': p_value,
        'std_err': std_err,
        'n': len(pop)
    }
    
    # Criar linha de regressão
    log_pop_fit = np.linspace(log_pop.min(), log_pop.max(), 100)
    log_hom_fit = intercept + slope * log_pop_fit
    
    # Plot dos dados e linha de regressão
    axes[idx].loglog(pop, hom, 'o', alpha=0.5, markersize=4, label='Dados')
    axes[idx].loglog(10**log_pop_fit, 10**log_hom_fit, 'r-', linewidth=2, 
                     label=f'β = {slope:.3f}')
    
    axes[idx].set_xlabel('População (log)', fontsize=10)
    axes[idx].set_ylabel('Homicídios (log)', fontsize=10)
    axes[idx].set_title(f'Ano {year}\\nβ = {slope:.3f}, R² = {r_value**2:.3f}\\n(n={len(pop)} municípios)', 
                       fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3, which='both')
    axes[idx].legend(loc='upper left', fontsize=9)

# Remover subplot extra
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

# Exibir tabela com resultados
print("\nResultados da Regressão Linear (Log-Log):")
print("="*80)
print(f"{'Ano':<8} {'Beta (β)':<12} {'Alpha (α)':<12} {'R²':<10} {'p-value':<12} {'n':<8}")
print("-"*80)
for year in years:
    res = regression_results[year]
    print(f"{year:<8} {res['beta']:<12.4f} {res['alpha']:<12.4f} {res['r_squared']:<10.4f} {res['p_value']:<12.2e} {res['n']:<8}")
print("="*80)

## 7. Evolução Temporal do Expoente β

Análise da variação do expoente β (escala) ao longo dos anos para municípios com população ≥ 10.000 habitantes.

In [ ]:
# Extrair valores de beta para cada ano
beta_values = [regression_results[year]['beta'] for year in years]
r_squared_values = [regression_results[year]['r_squared'] for year in years]

# Criar figura com dois subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Beta vs Ano
ax1.plot(years, beta_values, 'o-', markersize=10, linewidth=2, color='darkblue')
ax1.set_xlabel('Ano', fontsize=12, fontweight='bold')
ax1.set_ylabel('Expoente β (Slope)', fontsize=12, fontweight='bold')
ax1.set_title('Evolução Temporal do Expoente β\n(Relação Homicídios vs População)', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(years)

# Adicionar valores de beta nos pontos
for year, beta in zip(years, beta_values):
    ax1.annotate(f'{beta:.3f}', 
                xy=(year, beta), 
                xytext=(0, 10),
                textcoords='offset points',
                ha='center',
                fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))

# Adicionar linha de referência β=1 (linear)
ax1.axhline(y=1.0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='β = 1 (linear)')
ax1.legend()

# Plot 2: R² vs Ano (qualidade do ajuste)
ax2.plot(years, r_squared_values, 's-', markersize=10, linewidth=2, color='darkgreen')
ax2.set_xlabel('Ano', fontsize=12, fontweight='bold')
ax2.set_ylabel('R² (Coeficiente de Determinação)', fontsize=12, fontweight='bold')
ax2.set_title('Qualidade do Ajuste ao Longo do Tempo', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xticks(years)
ax2.set_ylim([0, 1])

# Adicionar valores de R² nos pontos
for year, r2 in zip(years, r_squared_values):
    ax2.annotate(f'{r2:.3f}', 
                xy=(year, r2), 
                xytext=(0, 10),
                textcoords='offset points',
                ha='center',
                fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.5))

plt.tight_layout()
plt.show()

# Análise estatística da tendência
print("\nAnálise da Evolução de β:")
print("="*60)
print(f"β médio: {np.mean(beta_values):.4f}")
print(f"β mínimo: {np.min(beta_values):.4f} (ano {years[np.argmin(beta_values)]})")
print(f"β máximo: {np.max(beta_values):.4f} (ano {years[np.argmax(beta_values)]})")
print(f"Variação total: {np.max(beta_values) - np.min(beta_values):.4f}")
print(f"Desvio padrão: {np.std(beta_values):.4f}")
print("="*60)

# Interpretação do expoente β
print("\nInterpretação:")
print("-"*60)
if np.mean(beta_values) > 1:
    print(f"β > 1: Relação SUPERLINEAR - Cidades maiores têm")
    print(f"       proporcionalmente mais homicídios per capita")
elif np.mean(beta_values) < 1:
    print(f"β < 1: Relação SUBLINEAR - Cidades maiores têm")
    print(f"       proporcionalmente menos homicídios per capita")
else:
    print(f"β ≈ 1: Relação LINEAR - Homicídios crescem")
    print(f"       proporcionalmente com a população")
print("-"*60)